In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('./train.csv'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
#
## You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### **Hi! We will go through how a basic convolutional neural network works in tensorflow. If you find this helpful/insightful please do upvote this notebook :)**

### **thank you!**

## **Loading the training and test datasets:**
The training and test datasets are specified as 'emnist-letters-train.csv' and 'emnist-letters-test.csv' respectively. We will read these load these read these files using pandas

In [3]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [4]:
train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## **Exploring the data:**
Although we already know that we are dealing with a letters dataset, it's better to see what our data looks like:

In [6]:
print('training dataset dimensions: ', train.shape)
print('test dataset dimensions: ', test.shape, '\n')

training dataset dimensions:  (42000, 785)
test dataset dimensions:  (28000, 784) 



From here we can see that we are dealing with a training with 88799 examples and a test dataset having  14799 examples, both having 784 pixels per data (the first column being the label). Our plan is to **create a validation split from the training data and then use the test set for evaluating how well can the neural network predict new data**

In [7]:
# view the head of the training dataset
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


From the initial view of the training dataset, the first column contains the labels and the rest of the columns are the pixles. let's just fix the column names to properly label each cell:

In [10]:
# update column names for both datasets
columns = ['labels']
for i in range(train.shape[1]-1):
    columns.append(i)
    
train.columns = columns

In [11]:
train

,labels,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#test.columns = columns , commenter parceque test n'a pas la colonne label

classes = train['labels'].unique()

In [21]:
classes

array([  0, 254, 253,  33, 240, 201,  88,  27,  73,  21, 128, 252, 198,
       165, 168,  46, 127,  29,  71,  17,  68, 188,  36, 255, 191,  38,
       164, 169, 133, 242, 246, 149, 101, 244, 235,  60,  93, 117, 185,
       162, 248,  13, 111, 232, 200,  25, 106,  12, 193,  65, 160, 199,
        50,  56, 231, 230,  78, 167, 170, 251, 131,  96,  66, 223, 207,
        84, 208,  48,  58, 206, 203,  10,  87,  11,  63,  61,  75,  24,
        85,  64, 217, 212,  28,  19,   3,  16,  81, 186,   4, 108,  69,
       121, 236, 192, 195, 204,   8, 129, 233, 184,  42, 158, 148, 152,
       247, 159, 218, 115, 228,  79, 135,   1,  35,  18, 238, 132, 245,
       250, 205, 202, 124, 182, 122, 142, 134, 197, 211,  76,  74,  45,
       139,  22, 166, 118, 249,  14,  43, 189, 151, 225,  80, 214, 154,
        57, 143, 156, 187,  72,  15,  89,  47, 174,   2,  39, 102, 126,
        62,  97, 163, 237,  86,  92, 104, 243,  67, 150, 221, 103, 216,
       227,  31,  83, 157, 141, 181,  49, 110, 239, 155, 196, 20

In [22]:
print('number of classes: ', len(classes))

train.head()

number of classes:  256


,labels,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The 26 classes here correspond to the 26 unique characters from the english alphabet

## **Creating a training and validation split**
Now we'll create a random train and validation dataset split. This means that **a sample from the training data will be used to evaluate the performance the our neural network while adjusting its hyperparameters**. After splitting the traning and validation set randomly, we will then reshape and normalize the inputs to make it compatible with the input requirements of the neural network

In [ ]:
from sklearn.model_selection import train_test_split

# split training and validation data using sklearn
x_train, x_val, y_train, y_val = train_test_split(train.drop(['labels'], axis=1),
                                                  train.labels - 1,
                                                  train_size=0.8,
                                                  test_size=0.2,
                                                  random_state=42)

# reshape and normalize test data
x_train = x_train / 255.0
x_val = x_val / 255.0

testX = test.values[:, 1:].reshape(test.shape[0],28, 28, 1).astype('float32')
x_test = testX / 255.0
y_test = test['labels'].values - 1 # this is just to make the neurons in the output layer start at 0

print('trianing set: ', x_train.shape, y_train.shape)
print('validation set: ', x_val.shape, y_val.shape)
print('test set: ', x_test.shape, y_test.shape)

## **Viewing the sample images of the training and test set**
Using matplotlib, let's plot some of the images from the training dataset. This will print out random samples everytime we run the code

In [ ]:
import matplotlib.pyplot as plt
import random
# select random samples from training dataset
train_samples = random.sample(range(0, len(x_train)), 9)
test_samples = random.sample(range(0, len(x_val)), 9)

x_train
plt.figure(figsize=(6, 6))
plt.suptitle('Training set')
for i in train_samples:
    plt.subplot(3, 3, train_samples.index(i)+1)
    plt.imshow(x_train.iloc[i,:].values.reshape(28,28), cmap='binary')
    plt.title(f'label: {y_train.iloc[i]}')
    plt.axis('off')
    
plt.figure(figsize=(6, 6))
plt.suptitle('Test set')
for i in test_samples:
    plt.subplot(3, 3, test_samples.index(i)+1)
    plt.imshow(x_val.iloc[i,:].values.reshape(28,28), cmap='binary')
    plt.title(f'label: {y_val.iloc[i]}')
    plt.axis('off')

### **Creating the CNN Model**
Since we have an idea of what our data is, we can finally start creating our neural network

In [ ]:
import tensorflow as tf

# set accuracy
desired_accuracy = 0.92
    
# create callback to stop training when we reached desired accuracy
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') is not None and logs.get('accuracy') >= desired_accuracy):
            print('\nReached 92% training accuracy: cancelling training...')
            self.model.stop_training = True

# instantiate callback
callbacks = myCallback()
    

def train_model():
    # define model
    model = tf.keras.models.Sequential([
        # initial normalization
        tf.keras.layers.Reshape((28, 28, 1), input_shape=(784,)),
#         tf.keras.layers.BatchNormalization(),
        
        # first convolution
        tf.keras.layers.Conv2D(8, (3, 3), activation='relu'), # applies kernels to our data
        tf.keras.layers.MaxPooling2D(2, 2), # reduce dimension
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.4),
        
        # second convolution
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.4),
        
        # third convolution
        tf.keras.layers.Conv2D(24, (3, 3), activation='relu'),
#         tf.keras.layers.MaxPooling2D(2, 2),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.4),

    
        # feed to DNN
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(len(classes), activation=tf.nn.softmax) # generalized logistic regression
    ])
    
    # use sparse categorical crossentropy since values are labeled from 0-25
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

### **What have we implemented/experimented in this neural network so far?**
We have the following layers in our CNN:
* **Convolutional layers** - what convolutional layers do is that it applies various kernels on the image to extract or identify important features that we will feed into the neural network

* **Pooling filters** - pooling helps reduce the spatial size of an image to reduce the amount of parameters when computing for the output. Progressively reducing the dimensions (down sampling) of the images/feature representations in each convolution will later help reducing the computational cost of the neural network

We also have omitted the following:
* **Droputs** -  dropouts ignore a percentage of the 'neurons' in each layers. Doing this helps in preventing overfitting the training data

* **Batch normalization**  - this helps in  re-centering and re-scaling our outputs in every layer

**NOTE:**

 At this point we've already repetitively tuned the hyperparameters to get the desired result from the neural network. That is why we've omitted normalizing and dropouts in the model
 
 
 
 **What are these so-called hyperparameters?**
 
these are the what we change in our neural network to optimize its performance. Some include the number of neurons in a layer, what activation function we will use, the optimizer, learning rate, batch size, and epochs.

In [ ]:
# view model summary before running neural network
model = train_model()
model.summary()


## **Training the neural network:**
So far we've defined some the parameters of our neural network . We also created a training and validation split. Now we'll use these data to train our model

In [ ]:
# train neural network and have it automatically stop on 95% accuracy
history = model.fit(x_train, y_train, epochs=1000,
                    validation_data=(x_val, y_val),
                    batch_size=4096, verbose=1,
                    callbacks=[callbacks])

**After a few epochs our neural network has a reached ~92% accuracy in the training set and a ~90% accuracy in the validation set**. Adding the dropouts will make the accuracy of the validation test greater than the training set. Dropouts will ignore a percentage of the neurons in the specified layer which, in return, can help avoid overfitting and help the neural network generalize better. In our case, the results are already acceptable without droputs.

## **Visualizing the convolution and pooling filters**
This visualization will let us see the features that our neural network learned as it passed through the convolutions. **Observe that specific parts of the letter lights up** in each layer. We can interpret this as the network identifying those features of the letter

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

number = random.sample(range(0, len(x_val)), 1)[0]

# Run the image through the network and return the intermediate representations for the data.
successive_feature_maps = visualization_model.predict(x_val.iloc[number, :].to_numpy().reshape(1, 784))

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers[1:]]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 10. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

**By visualizing each convolution and pooling layer, we will have an idea of what 'features' are being learned by our neural network**. For example, it may be trying to identify specific edges and curves, or even 'holes' for specific letters. Pooling helps down sample these features so our neural network will be able to train faster

## **Plotting the losses and the accuracy of the training and validation set**
Let's see how our model performs as we increase the number of epochs

In [ ]:
# Plot training vs validation accruacy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('CNN Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Number of Epochs')
plt.legend(['training', 'validation'], loc='lower right')
plt.show()

# Plot training vs validation losses
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('CNN Model Loss')
plt.ylabel('Loss')
plt.xlabel('Number of Epochs')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

Plotting the accuracy and loss over time gives us an idea well our neural network is performing for each epoch. **We can also use these graphs to identify at what epoch can we stop training** to avoid overtraining our network. For example I've set the number of epochs in this network to be 1000. Viewing the plot can help us identify which epoch to stop

In relation to this, **we can also set our network to automatically stop training at a given training accuracy**. This is done by using **callbacks**.

## **Neural network evaluation**
Here we will evaluate the performace of our neural network by using the test set. Based on the output, there will be instances that the predicted will be different from the actual label.

In [ ]:
evaluate = model.evaluate(x_test, y_test)

# predict values for all data
predictions = model.predict(x_test)
print("predictions shape:", predictions.shape)


### **What does our evaluation accuracy mean?**
* If our training set is significantly higher that our validation set, it means that we are overfitiing our training data. This will result to our neural network performing poorly in our validation and test data. 

* In our case, however, we can see that our test accuracy is farily close with our training and  validation accuracy. This means that our CNN generalized well with our test data.

In [ ]:
# select random data from test set and predict it's classification
test_samples = random.sample(range(0, len(x_test)), 30) # select 30 samples

# display results
plt.figure(figsize=(15, 6))
for i in test_samples:
    plt.subplot(3, 10, test_samples.index(i) + 1)  
    plt.imshow(x_test[i].reshape(28,28), cmap='magma')
    plt.title(f'predicted: {np.argmax(predictions[i])} \nactual: {y_test[i]}')
    plt.axis('off')

#### **We can now see the what our model predicted vs what the actual label is. Depending on the sample displayed, we expect that there will be some misclassifications since our accuracy isn't 100%**

#### **Our labels are defined from 0 (a or A) to 25 (z or Z)**



### ***Thank you for checking this notebook out!***